In [1]:
from google.colab import drive, files
drive.mount("/content/drive")                 # 左のポップアップで許可を与える

# ------------ ここを自分の配置に合わせて確認 -------------
!ls -1 "/content/drive/MyDrive" | head


Mounted at /content/drive
AffectNet
AU.ipynb
ckpts
Colab Notebooks
diffusion.ipynb
image_rename.ipynb
poo.jpg
practice.ipynb
report3_202312944_uno.ipynb
wine_linearRegression_scaling.ipynb


In [2]:
# ────────────── Cell 1 : 画像 ↔ AU ベクトルのペア一覧を生成 ──────────────
import json, csv, pandas as pd
from pathlib import Path

ROOT = Path("/content/drive/MyDrive/AffectNet")

IMG_DIR = {                         # 画像
    "Train": ROOT / "Train",
    "Test" : ROOT / "Test",
}
AU_DIR  = {                         # OpenFace が出力した AU
    "Train": ROOT / "OpenFace_AU_Train",
    "Test" : ROOT / "OpenFace_AU_Test",
}
OUT_CSV = {s: ROOT / f"{s.lower()}_pairs.csv" for s in ("Train", "Test")}

def load_expr_au(split:str, expr:str):
    """
    anger.csv などを読み、
    frame(1…) -> AU ベクトル(list[float]) の dict を返す
    """
    csv_path = AU_DIR[split] / expr / f"{expr.lower()}.csv"
    if not csv_path.exists():       # Contempt.csv など大文字パターンも試す
        csv_path = AU_DIR[split] / expr / f"{expr.capitalize()}.csv"
    if not csv_path.exists():
        return {}
    df = pd.read_csv(csv_path)
    au_cols = [c for c in df.columns if c.startswith("AU")]
    # frame 列は 1,2,… または 0,1,… の場合があるのでそのまま key に
    return {int(r["frame"]): r[au_cols].astype(float).tolist()
            for _, r in df.iterrows()}

def build_pairs(split:str):
    rows = []
    for expr_dir in IMG_DIR[split].iterdir():          # anger / happy …
        if not expr_dir.is_dir(): continue
        expr = expr_dir.name.lower()
        au_table = load_expr_au(split, expr)
        if not au_table:
            print(f"[warn] AU csv missing → {AU_DIR[split]/expr}")
            continue

        for img_path in sorted(expr_dir.glob("image_*.jpg")):
            idx = int(img_path.stem.split("_")[1])     # image_00023 → 23
            # OpenFace frame は 1-or-0 start ⇒ 両方試す
            au_vec = au_table.get(idx) or au_table.get(idx-1)
            if au_vec is None:
                continue
            rows.append({"image_path": str(img_path),
                         "au_vector" : json.dumps(au_vec)})

    OUT_CSV[split].parent.mkdir(parents=True, exist_ok=True)
    with open(OUT_CSV[split], "w", newline="") as f:
        w = csv.DictWriter(f, fieldnames=["image_path", "au_vector"])
        w.writeheader(); w.writerows(rows)
    print(f"{split}: {len(rows):,} pairs → {OUT_CSV[split]}")

# 実行
build_pairs("Train")
build_pairs("Test")


Train: 11,183 pairs → /content/drive/MyDrive/AffectNet/train_pairs.csv
Test: 8,733 pairs → /content/drive/MyDrive/AffectNet/test_pairs.csv
